In [ ]:
# SQL en Python

In [116]:
import sqlite3

In [4]:
type(sqlite3)

module

In [6]:
conn= sqlite3.connect("first.db")

In [8]:
conn

In [10]:
conn.cursor()

In [12]:
cur= conn.cursor()

In [14]:
cur.execute("""
    CREATE TABLE ice_cream_flavors (
        ID INTEGER PRIMARY KEY,
        Flavor TEXT,
        Rating INTEGER
        );
""")

In [18]:
# Inserción de valores
cur.execute("""
    INSERT INTO ice_cream_flavors(Flavor, Rating)
    VALUES ('choco', 10), ('straw',9),('pistachio',7.2);
""")
conn.commit()

In [22]:
cur.execute("SELECT * FROM ice_cream_flavors;")

In [24]:
cur.fetchone()

(1, 'choco', 10)

In [26]:
cur.fetchone()

(2, 'straw', 9)

In [28]:
next(cur)

(3, 'pistachio', 7.2)

In [32]:
cur.execute("SELECT * FROM ice_cream_flavors;")
for row in cur:
    print(row)

(1, 'choco', 10)
(2, 'straw', 9)
(3, 'pistachio', 7.2)


In [34]:
cur.execute("SELECT * FROM ice_cream_flavors;")
cur.fetchmany(2)

[(1, 'choco', 10), (2, 'straw', 9)]

In [36]:
cur.fetchmany(2)

[(3, 'pistachio', 7.2)]

In [38]:
# Ejemplo de una mala ejecución
cur.execute("SELECT * FROM ice_cream_flavors;")
result= cur.fetchone()
result['Flavor']

TypeError: tuple indices must be integers or slices, not str

In [40]:
cur.row_factory = lambda cursor, row: dict(zip([col[0] for  col in cursor.description],row))

In [42]:
cur.execute("SELECT * FROM ice_cream_flavors;")
result= cur.fetchone()
result['Flavor']

'choco'

In [44]:
result['Rating']

10

In [46]:
type(result)

dict

In [48]:
cur.execute("SELECT * FROM ice_cream_flavors;")
result= cur.fetchall()
result

[{'ID': 1, 'Flavor': 'choco', 'Rating': 10},
 {'ID': 2, 'Flavor': 'straw', 'Rating': 9},
 {'ID': 3, 'Flavor': 'pistachio', 'Rating': 7.2}]

In [50]:
# Camino alternativo
cur.row_factory= sqlite3.Row
cur.execute("SELECT * FROM ice_cream_flavors;")
result= cur.fetchall()
result

In [52]:
first_row= result[0]
print(first_row["Flavor"])
print(first_row["Rating"])

choco
10


In [54]:
first_row= result[1]
print(first_row["Flavor"])
print(first_row["Rating"])

straw
9


In [56]:
type(first_row)

sqlite3.Row

In [ ]:
# Si es en un mismo renglón, no se necesitan las 3 comillas (hasta que no se use conn.commit(), la instrucción no se ve reflejada)

In [58]:
cur.execute("INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES ('hazelnut', 6);")

In [60]:
# si no se ha usado conn.commit(), se puede cancelar la instrucción

In [64]:
conn.rollback()

In [66]:
cur.execute("INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES ('nut', 8);")
conn.commit()

In [68]:
# Parte 2

In [74]:
# Introducir nuevos datos a la tabla con inputs
Flavor= input("introduzca un sabor")
Rating= input("introduzca una calificación")
cur.execute(f"INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES ('{Flavor}','{Rating}');")
conn.commit()

introduzca un sabor DVD
introduzca una calificación 23


In [80]:
# Modo alternativo
Flavor= input("introduzca un sabor")
Rating= input("introduzca una calificación")
cur.execute(f"INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES (?,?);",(Flavor,Rating))
conn.commit()

introduzca un sabor Cereza
introduzca una calificación 9.5


In [84]:
# Modo alternativo 2

Flavor= input("introduzca un sabor")
Rating= input("introduzca una calificación")
cur.execute(f"INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES (:sabor,:puntaje);",{"sabor":Flavor, "puntaje":Rating})
conn.commit()

introduzca un sabor banana
introduzca una calificación 8.3


In [86]:
ice_cream_flavors=[("vainilla",4.5),
                   ("fresa",6.7),
                   ("café",3.4),
                   ("caramelo",9),
                  ]

In [88]:
cur.executemany(f"INSERT INTO ice_cream_flavors(Flavor, Rating) VALUES(?,?);", ice_cream_flavors)
conn.commit()

In [92]:
# Creación de una nueva tabla mediante un script (TODO EN UN MISMO CÓDIGO A DIFERENCIA DEL PRIMER CASO)
cur.executescript("""
    CREATE TABLE IF NOT EXISTS toppings(
        ID INTEGER PRIMARY KEY,
        name TEXT NOT NULL);
    INSERT INTO toppings (name) VALUES ('sprinkles'),('choco sauce'), ('caramel sauce'), ('nuts');
""")

In [94]:
conn.commit()

In [96]:
# DATABASE DUMP ( Te muestra el código de SQL que se requirió)
for line in conn.iterdump():
    print(line)

BEGIN TRANSACTION;
CREATE TABLE ice_cream_flavors (
        ID INTEGER PRIMARY KEY,
        Flavor TEXT,
        Rating INTEGER
        );
INSERT INTO "ice_cream_flavors" VALUES(1,'choco',10);
INSERT INTO "ice_cream_flavors" VALUES(2,'straw',9);
INSERT INTO "ice_cream_flavors" VALUES(3,'pistachio',7.2);
INSERT INTO "ice_cream_flavors" VALUES(4,'nut',8);
INSERT INTO "ice_cream_flavors" VALUES(5,'DVD',23);
INSERT INTO "ice_cream_flavors" VALUES(6,'Cereza',9.5);
INSERT INTO "ice_cream_flavors" VALUES(7,'banana',8.3);
INSERT INTO "ice_cream_flavors" VALUES(8,'vainilla',4.5);
INSERT INTO "ice_cream_flavors" VALUES(9,'fresa',6.7);
INSERT INTO "ice_cream_flavors" VALUES(10,'café',3.4);
INSERT INTO "ice_cream_flavors" VALUES(11,'caramelo',9);
CREATE TABLE toppings(
        ID INTEGER PRIMARY KEY,
        name TEXT NOT NULL);
INSERT INTO "toppings" VALUES(1,'sprinkles');
INSERT INTO "toppings" VALUES(2,'choco sauce');
INSERT INTO "toppings" VALUES(3,'caramel sauce');
INSERT INTO "toppings" VALU

In [98]:
# GENERAR DOCUMENTO CON EL CÓDIGO ("w" instrucción tipo write)
_file= open("ice_cream_flavors","w")

for line in conn.iterdump():
    _file.write(f"{line}\n")

_file.close()

In [104]:
# OTRO MÉTODO DE GENERAR DOCUMENTO CON EL CÓDIGO ("w" instrucción tipo write)
with open("ice_cream_db_dumb2.sql","w") as _file:
    for line in conn.iterdump():
        _file.write(f"{line}\n")


In [106]:
# CONTEXT MANAGERS, OTRA HERRAMIENTA PARA TRABAJAR LAS BASES ( TE AHORRAS EL conn.commit)

with conn:
    conn.execute("INSERT INTO ice_cream_flavors(Flavor,Rating) VALUES(?,?);",
                 ("caramelo extremo",2.7))

In [108]:
# SE PUEDEN SELECCIONAR ALGUNOS DATOS DE LA TABLA PONIENDO UN LÍMITE

conn.execute("SELECT* FROM ice_cream_flavors LIMIT 4;").fetchall()

[(1, 'choco', 10), (2, 'straw', 9), (3, 'pistachio', 7.2), (4, 'nut', 8)]

In [110]:
#  CERRAR CONEXIÓN

conn.close()

In [112]:
# Abrir conexión

conn= sqlite3.connect("first.db")

In [114]:
conn.close()